# Summarize skill results
***

**Author**: Chus Casado Rodríguez<br>
**Date**: 16-06-2023<br>


**Introduction**:<br>
This notebook creates a table comparing the skill of the diverse notification criteria optimized for different f-scores.

In [1]:
import os
path_root = os.getcwd()
import glob
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime, timedelta
import pickle
import yaml

import warnings
warnings.filterwarnings("ignore")

os.chdir('../py/')
from compute import hits2skill
os.chdir(path_root)

## 1 Configuration

In [2]:
with open("../conf/config.yml", "r", encoding='utf8') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)

### 1.1 Reporting points

In [3]:
# area threshold
area_threshold = cfg.get('reporting_points', {}).get('area', 500)

# reporting points
path_stations = cfg.get('reporting_points', {}).get('output', '../results/reporting_points/')
file_stations = f'{path_stations}reporting_points_over_{area_threshold}km2.parquet'

# catchments
catchments = cfg.get('reporting_points', {}).get('catchments', None)

# minimum performance required from the reporting points
min_kge = cfg.get('reporting_points', {}).get('KGE', None)

### 1.2 Hits

In [4]:
# parameters of the rolling window used to compute hits
window = cfg.get('hits', {}).get('window', 1)

# dissagregate the analysis by seasons?
seasonality = cfg.get('hits', {}).get('seasonality', False)

# path that contains the NetCDFs with hit, misses and false alarms pro
path_in = cfg.get('hits', {}).get('output', '../results/hits/')
path_in = f'{path_in}{window}/'

### 1.3 Skill

In [5]:
# current operationa criteria
current_criteria = cfg.get('skill', {}).get('current_criteria', None)

# fixed notification criteria
min_leadtime = cfg.get('skill', {}).get('leadtime', 60) 
min_area = cfg.get('skill', {}).get('area', 2000) 

# coefficient of the fbeta-score
betas = [0.8, 1, 1.2]

# path where results will be saved
path_out = cfg.get('skill', {}).get('output', f'../results/skill/')
if min_kge is not None:
    path_out = f'{path_out}kge_{min_kge}/'

## 2 Data

### 2.1 Reporting points

I load all the stations that where selected in a previous [notebook](3_0_select_stations.ipynb).

In [6]:
# load table of fixed reporting points
stations = pd.read_parquet(file_stations)
stations[['X', 'Y', 'area']] = stations[['X', 'Y', 'area']].astype(int)

# select stations that belong to the selected catchments
if catchments is not None:
    if isinstance(catchments, list) is False:
        catchments = [catchments]
    stations = stations.loc[stations.catchment.isin(catchments),:]

# remove points with a performance (KGE) lower than the established threshold
if min_kge is not None:
    mask_kge = ~(stations.KGE <= min_kge)
    stations = stations.loc[mask_kge]
else:
    # remove station with erroneous behaviour
    stations = stations.loc[~(stations.n_events_obs >= 6)]

# mask stations with events
stations_w_events = (stations.n_events_obs > 0)

print('All points')
print('----------')
print(f'no. reporting points:\t\t{stations.shape[0]}')
print('no. stations with events:\t{0}'.format(stations_w_events.sum()))
print('no. observed events:\t\t{0}'.format(stations.n_events_obs.sum()))

# select stations according to catchment area
if min_area > area_threshold:
    stations_optimize = stations.loc[stations.area >= min_area].index
else:
    stations_optimize = stations.index

print('\nPoints selected for otimization')
print('-------------------------------')
print(f'no. reporting points:\t\t{len(stations_optimize)}')
print('no. stations with events:\t{0}'.format((stations.loc[stations_optimize, 'n_events_obs'] > 0).sum()))
print('no. observed events:\t\t{0}'.format(stations.loc[stations_optimize, 'n_events_obs'].sum()))

# suffix that will be used when saving plots
suffix = f'{min_area}km2_{len(stations_optimize)}points'

All points
----------
no. reporting points:		1979
no. stations with events:	831
no. observed events:		1264

Points selected for otimization
-------------------------------
no. reporting points:		1239
no. stations with events:	480
no. observed events:		678


### 2.2 Hits, misses and false alarms

In [7]:
# import hits for each station
hits_stn = xr.open_mfdataset(f'{path_in}*.nc', combine='nested', concat_dim='id')
# reorder persistences
hits_stn = hits_stn.sel(persistence=['1/1', '2/4', '2/3', '2/2', '3/4', '3/3'])
# extract selected stations
hits_stn = hits_stn.sel(id=stations.index.to_list()).compute()

# convert to NaN values at long leadtimes for which the persistence criteria is impossible to meet
hits_stn = hits_stn.astype(float)
for persistence in hits_stn.persistence.data:
    last_leadtime = int(persistence.split('/')[0]) - 1
    if last_leadtime > 0:
        hits_stn.sel(persistence=persistence)[dict(leadtime=slice(-last_leadtime, None))] = np.nan

# subset of the 'hits' dataset with the stations selected for the optimization
hits_opt = hits_stn.sel(id=stations_optimize).sum('id', skipna=False)

## 3 Analysis

In this section I will compute the skill of the EFAS predictions in different ways. In all the following sections I will work with three metrics: $recall$, $precision$ and the $f_{beta}$ score. The three metrics are based in the contingency table of hits ($TP$ for true positives), false alarms ($FP$ for false positives) and misses ($FN$ for false negatives).

$$recall = \frac{TP}{TP + FN}$$
$$precision = \frac{TP}{TP + FP}$$
$$f_{beta} = \frac{(1 + \beta^2) \cdot TP}{(1 + \beta^2) \cdot TP + \beta^2 \cdot FN + FP}$$


### 3.2 Compare approaches
#### 3.2.1 Import optimize criteria

In [9]:
criteria = {'current': current_criteria}
for beta in betas:
    metric = f'f{beta}'
    file = glob.glob(f'{path_out}{metric}/*{suffix}.pkl')[0]
    opt_crit = pickle.load(open(file, 'rb'))
    for app, crit in opt_crit.items():
        if metric in crit:
            del crit[metric]
        criteria[f'{metric}_{app}'] = crit

#### 3.2.2 Compare approaches

In [ ]:
# transform criteria into a DataFrame
summary_criteria = pd.concat([pd.DataFrame(crtr, index=[i]) for i, crtr in criteria.items()], axis=0)
summary_criteria.approach = [''.join([x[0].upper() for x in app.split('_')]) for app in summary_criteria.approach]
summary_criteria['OF'] = [x.split('_')[0] if x != 'current' else '' for x in summary_criteria.index]
summary_criteria = summary_criteria[['approach', 'OF', 'probability', 'persistence']]

In [ ]:
# compute hits, misses and false alarms
summary_hits = pd.DataFrame({i: hits_opt.sel(leadtime=min_leadtime).sel(crtr).to_pandas() for i, crtr in criteria.items()}).transpose()
summary_hits = summary_hits.astype(int)
summary_hits['no_events'] = summary_hits.TP + summary_hits.FN

In [ ]:
# compute skill
summary_hits['recall'] = summary_hits.TP / (summary_hits.TP + summary_hits.FN)
summary_hits['precision'] = summary_hits.TP / (summary_hits.TP + summary_hits.FP)
for beta in betas:
    summary_hits[f'f{beta}'] = (1 + beta**2) * summary_hits.TP / ((1 + beta**2) * summary_hits.TP + beta**2 * summary_hits.FN + summary_hits.FP)

In [ ]:
# concat criteria, hits and summary data frames
summary = pd.concat((summary_criteria, summary_hits), axis=1)
summary.sort_values('approach', inplace=True)

summary

In [ ]:
# export
summary.to_csv(f'{path_out}skill_by_criteria.csv', float_format='%.3f', index=False)

***

In [8]:
hits_stn

<xarray.Dataset>
Dimensions:      (id: 1979, persistence: 6, approach: 4, leadtime: 20, probability: 37)
Coordinates:
  * id           (id) int64 1 2 3 4 5 8 9 ... 5224 5225 5226 5228 5230 5231 5232
  * leadtime     (leadtime) int32 12 24 36 48 60 72 ... 180 192 204 216 228 240
  * probability  (probability) float64 0.05 0.075 0.1 0.125 ... 0.9 0.925 0.95
  * approach     (approach) object '1_deterministic_+_1_probabilistic' ... 'b...
  * persistence  (persistence) object '1/1' '2/4' '2/3' '2/2' '3/4' '3/3'
Data variables:
    TP           (id, persistence, approach, leadtime, probability) float64 0...
    FN           (id, persistence, approach, leadtime, probability) float64 0...
    FP           (id, persistence, approach, leadtime, probability) float64 3...

In [17]:
hits_1D1P = hits_stn.sel(approach='1_deterministic_+_1_probabilistic',
                         probability=0.375,
                         persistence='1/1',
                         leadtime=min_leadtime)

In [18]:
hits_BW = hits_stn.sel(approach='brier_weighted',
                       probability=0.375,
                       persistence='1/1',
                         leadtime=min_leadtime)

In [20]:
hits_diff = hits_BW - hits_1D1P

In [23]:
tp = hits_diff['TP'].to_pandas()

In [25]:
tp[tp == 0]

id
1       0.0
2       0.0
3       0.0
4       0.0
5       0.0
       ... 
5225    0.0
5228    0.0
5230    0.0
5231    0.0
5232    0.0
Length: 1909, dtype: float64

In [26]:
tp[tp < 0]

id
135    -1.0
257    -1.0
595    -1.0
616    -1.0
644    -1.0
841    -1.0
1014   -1.0
1091   -1.0
1228   -1.0
1322   -1.0
1503   -2.0
1664   -1.0
2320   -1.0
2364   -1.0
4492   -1.0
4578   -1.0
dtype: float64

In [31]:
stations[tp > 0].value_counts('river')

river
Inn                        4
Cele                       2
Clyde                      2
Danube                     2
Tiber                      2
Main                       2
Altaelva                   1
Salzach                    1
Oslawa                     1
Oudon                      1
RIONI                      1
Raska                      1
Rivera de los limonetes    1
Ruhr                       1
Saalach                    1
Sventoji                   1
Sauer                      1
Supsa                      1
Oise                       1
TAMAR                      1
TOLLENSE                   1
TORRIDGE                   1
Tagus                      1
Tengelionjoki              1
Orekilsalven               1
Motala Stroem              1
Narpion                    1
Munster Blackwater         1
Dessoubre                  1
Deveron                    1
Gaula                      1
Gaula (Melhus)             1
Glomma                     1
Isel                       1
Ivalo   

In [30]:
stations[tp > 0].value_counts('catchment')

catchment
Danube           12
Rhine             5
Altaelva          2
Clyde             2
Garonne           2
Gaula             2
Tiber             2
Seine             2
Supsa             1
Rhone             1
Rioni             1
Thames            1
Tagus             1
Tamar             1
Paatsjoki         1
Torne             1
Torridge          1
Peene             1
Nestos            1
Orekilsalven      1
Oder              1
Blackwater        1
Nemunas           1
Narpionjoki       1
Motala Stroem     1
Moscarello        1
Minho             1
Loire             1
Jucar             1
Guadiana          1
Glomma            1
Deveron           1
Weaver            1
dtype: int64

In [35]:
stations[tp > 0][stations.catchment == 'Rhine'].sort_values(['catchment', 'subcatchment', 'river'])

name        X        Y   area subcatchment  river catchment  \
id                                                                        
59     Steinbach  4297500  2992500  17925         Main   Main     Rhine   
486   Schwürbitz  4402500  3007500   2500         Main   Main     Rhine   
5226     Gendron  3962500  3022500   1425        Meuse  Lesse     Rhine   
4385    Michelau  4042500  2982500    925      Moselle  Sauer     Rhine   
934    Hattingen  4127500  3147500   4225         Ruhr   Ruhr     Rhine   

     country    KGE  correlation  ...  rl1.5    rl2    rl5    rl10    rl20  \
id                                ...                                        
59        DE  0.761        0.855  ...  555.7  649.3  879.5  1031.9  1178.1   
486       DE  0.847        0.904  ...  155.8  177.2  229.8   264.7   298.1   
5226      BE    NaN          NaN  ...  121.6  143.5  197.4   233.1   267.4   
4385      LU  0.739        0.806  ...   72.9   90.1  132.3   160.2   187.0   
934       DE  0.901        0.923  ...  316.3  358.3  461.5   529.9   595.5   

        rl50   rl100   rl200   rl500  n_events_obs  
id                                                  
59    1367.4  1509.2  1650.5  1837.0             1  
486    341.4   373.8   406.1   448.8             1  
5226   311.7   345.0   378.1   421.8             1  
4385   221.6   247.6   273.5   307.7             2  
934    680.3   743.9   807.3   890.9             1  

[5 rows x 24 columns]